# LOGISTIC REGRESSION

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
print ("CURRENT TF VERSION IS [%s]" % (tf.__version__))

CURRENT TF VERSION IS [1.0.1]


## MNIST

In [2]:
mnist      = input_data.read_data_sets('data/', one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
print ("MNIST loaded")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST loaded


## CREATE GRAPH

In [3]:
x = tf.placeholder("float", [None, 784], name="INPUT") 
y = tf.placeholder("float", [None, 10], name="TARGET")
with tf.variable_scope("PARAMETERS"):
    W = tf.Variable(tf.zeros([784, 10]), name="WEIGHT")
    b = tf.Variable(tf.zeros([10]), name="BIAS")
# LOGISTIC REGRESSION MODEL
with tf.variable_scope("MODEL"):
    actv = tf.nn.softmax(tf.matmul(x, W) + b) 
# COST FUNCTION
with tf.variable_scope("COST"):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv), reduction_indices=1)) 
# OPTIMIZER
learning_rate = 0.01
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# ACCURACY
with tf.variable_scope("ACCURACY"):
    pred = tf.equal(tf.argmax(actv, 1), tf.argmax(y, 1))    
    accr = tf.reduce_mean(tf.cast(pred, "float"))
# INITIALIZER
init = tf.global_variables_initializer()

## TRAIN THE MODEL

In [4]:
training_epochs = 10
batch_size      = 100
display_step    = 2
# SESSION
sess = tf.Session()
sess.run(init)
# MINI-BATCH LEARNING
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(mnist.train.num_examples/batch_size)
    for i in range(num_batch): 
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        feeds = {x: batch_xs, y: batch_ys}
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch
    # DISPLAY
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        feeds_test = {x: mnist.test.images, y: mnist.test.labels}
        train_acc = sess.run(accr, feed_dict=feeds_train)
        test_acc = sess.run(accr, feed_dict=feeds_test)
        print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
               % (epoch, training_epochs, avg_cost, train_acc, test_acc))
print ("DONE")

Epoch: 000/010 cost: 1.174406635 train_acc: 0.820 test_acc: 0.853
Epoch: 002/010 cost: 0.550508163 train_acc: 0.850 test_acc: 0.882
Epoch: 004/010 cost: 0.463635457 train_acc: 0.850 test_acc: 0.891
Epoch: 006/010 cost: 0.423894678 train_acc: 0.940 test_acc: 0.898
Epoch: 008/010 cost: 0.399870407 train_acc: 0.890 test_acc: 0.901
DONE


## TO USE TENSORBOARD 

In [5]:
from IPython.display import clear_output, Image, display, HTML
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [6]:
show_graph(tf.get_default_graph().as_graph_def())